In [1]:
task = 3

In [2]:
# from acdc.TLACDCExperiment import TLACDCExperiment
import pickle
corr = pickle.load(open("results/acdc_3/another_final_edges.pkl", "rb"))

/Users/cybershiptrooper/src/interpretability/MATS/.iit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import iit.utils.index as index

In [4]:
index.TorchIndex(corr[0][0][1].as_index)

[:]

In [5]:
corr

[(('blocks.1.hook_resid_post', [:], 'blocks.1.attn.hook_result', [:, :, 2]),
  0.0747578777372837),
 (('blocks.1.hook_resid_post', [:], 'blocks.1.attn.hook_result', [:, :, 1]),
  0.09008875861763954),
 (('blocks.1.attn.hook_v', [:, :, 2], 'blocks.1.hook_v_input', [:, :, 2]),
  0.03919454291462898),
 (('blocks.1.attn.hook_v', [:, :, 1], 'blocks.1.hook_v_input', [:, :, 1]),
  0.061149172484874725),
 (('blocks.1.hook_v_input', [:, :, 2], 'blocks.0.hook_mlp_out', [:]),
  0.05490947142243385),
 (('blocks.1.hook_v_input', [:, :, 1], 'blocks.0.hook_mlp_out', [:]),
  0.03585419803857803),
 (('blocks.0.hook_mlp_in', [:], 'hook_embed', [:]), 0.15206416323781013)]

tracr graph

In [6]:
from circuits_benchmark.commands.build_main_parser import build_main_parser
from circuits_benchmark.utils.get_cases import get_cases
from circuits_benchmark.transformers.tracr_circuits_builder import build_tracr_circuits
from circuits_benchmark.transformers.acdc_circuit_builder import get_full_acdc_circuit
from iit_utils._acdc_utils import get_gt_circuit, build_acdc_circuit_from_list_corr
import iit_utils.correspondence as correspondence

args, _ = build_main_parser().parse_known_args(
    [
        "compile",
        f"-i={task}",
        "-f",
    ]
)

cases = get_cases(args)
case = cases[0]

In [7]:
tracr_output = case.build_tracr_model()

In [8]:
hl_ll_corr = correspondence.TracrCorrespondence.from_output(case, tracr_output)

In [9]:
hl_ll_corr

{TracrHLNode(name: blocks.0.mlp.hook_post,
  label: is_x_3,
  classes: 0,
  index: [:]
 ): {LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None)},
 TracrHLNode(name: blocks.1.attn.hook_result,
  label: frac_prevs_1,
  classes: 0,
  index: [:, :, 0, :]
 ): {LLNode(name='blocks.1.attn.hook_result', index=[:, :, 1:3, :], subspace=None)}}

In [10]:
def hook_name(loc, style) -> str:
        layer, attn_or_mlp, unit = loc
        assert attn_or_mlp in ["attn", "mlp"], ValueError(f"Unknown attn_or_mlp {attn_or_mlp}")
        if style == "tl":
            return f"blocks.{layer}.{attn_or_mlp}.{'hook_result' if attn_or_mlp == 'attn' else 'hook_post'}"
        elif style == "wrapper":
            return f"mod.blocks.{loc}.mod.{attn_or_mlp}.hook_point"
        else:
            raise ValueError(f"Unknown style {style}")

def idx(loc):
    _, attn_or_mlp, unit = loc
    if isinstance(unit, index.TorchIndex):
        return unit
    assert attn_or_mlp in ["attn", "mlp"], ValueError(f"Unknown attn_or_mlp {attn_or_mlp}")
    if attn_or_mlp == "attn":
        return index.Ix[:, :, unit, :]
    assert unit is None
    return index.Ix[[None]]

In [11]:
tracr_edges = list(tracr_output.graph.edges)
tracr_nodes = list(tracr_output.graph.nodes)
tracr_edges, tracr_nodes

([('select_2', 'frac_prevs_1'),
  ('is_x_3', 'frac_prevs_1'),
  ('indices', 'select_2'),
  ('tokens', 'is_x_3')],
 ['frac_prevs_1', 'select_2', 'is_x_3', 'indices', 'tokens'])

In [12]:
ll_edges = correspondence.make_edge_corr(tracr_edges, hl_ll_corr)
print([list(ll_edge) for ll_edge in ll_edges])

[['blocks.0.mlp.hook_post', [:], 'blocks.1.attn.hook_result', [:, :, 1:3, :]]]


In [13]:
tracr_labels_circuit, tracr_circuit, tracr_alignment = build_tracr_circuits(tracr_output.graph, tracr_output.craft_model, granularity="acdc_hooks")

In [14]:
list(tracr_circuit.edges)[5][0].name, list(tracr_circuit.edges)[5][0].index

('blocks.1.hook_q_input', 0)

In [15]:
list(tracr_circuit.edges)

[(hook_embed, blocks.0.hook_mlp_in),
 (hook_pos_embed, blocks.1.hook_q_input[0]),
 (hook_pos_embed, blocks.1.hook_k_input[0]),
 (blocks.0.hook_mlp_in, blocks.0.hook_mlp_out),
 (blocks.0.hook_mlp_out, blocks.1.hook_v_input[0]),
 (blocks.1.hook_q_input[0], blocks.1.attn.hook_q[0]),
 (blocks.1.hook_k_input[0], blocks.1.attn.hook_k[0]),
 (blocks.1.hook_v_input[0], blocks.1.attn.hook_v[0]),
 (blocks.1.attn.hook_q[0], blocks.1.attn.hook_result[0]),
 (blocks.1.attn.hook_k[0], blocks.1.attn.hook_result[0]),
 (blocks.1.attn.hook_v[0], blocks.1.attn.hook_result[0]),
 (blocks.1.attn.hook_result[0], blocks.1.hook_resid_post)]

In [16]:
full_circuit = get_full_acdc_circuit(2, 4)
list(full_circuit.nodes), list(full_circuit.edges)

([hook_embed,
  hook_pos_embed,
  blocks.0.hook_q_input[0],
  blocks.0.attn.hook_q[0],
  blocks.0.hook_k_input[0],
  blocks.0.attn.hook_k[0],
  blocks.0.hook_v_input[0],
  blocks.0.attn.hook_v[0],
  blocks.0.attn.hook_result[0],
  blocks.0.hook_q_input[1],
  blocks.0.attn.hook_q[1],
  blocks.0.hook_k_input[1],
  blocks.0.attn.hook_k[1],
  blocks.0.hook_v_input[1],
  blocks.0.attn.hook_v[1],
  blocks.0.attn.hook_result[1],
  blocks.0.hook_q_input[2],
  blocks.0.attn.hook_q[2],
  blocks.0.hook_k_input[2],
  blocks.0.attn.hook_k[2],
  blocks.0.hook_v_input[2],
  blocks.0.attn.hook_v[2],
  blocks.0.attn.hook_result[2],
  blocks.0.hook_q_input[3],
  blocks.0.attn.hook_q[3],
  blocks.0.hook_k_input[3],
  blocks.0.attn.hook_k[3],
  blocks.0.hook_v_input[3],
  blocks.0.attn.hook_v[3],
  blocks.0.attn.hook_result[3],
  blocks.0.hook_mlp_in,
  blocks.0.hook_mlp_out,
  blocks.1.hook_q_input[0],
  blocks.1.attn.hook_q[0],
  blocks.1.hook_k_input[0],
  blocks.1.attn.hook_k[0],
  blocks.1.hook_v_inp

In [17]:
gt_circuit = get_gt_circuit(hl_ll_corr, full_circuit)
list(gt_circuit.nodes), list(gt_circuit.edges)

([hook_embed,
  hook_pos_embed,
  blocks.0.hook_mlp_in,
  blocks.0.hook_mlp_out,
  blocks.1.hook_q_input[1],
  blocks.1.attn.hook_q[1],
  blocks.1.hook_k_input[1],
  blocks.1.attn.hook_k[1],
  blocks.1.hook_v_input[1],
  blocks.1.attn.hook_v[1],
  blocks.1.attn.hook_result[1],
  blocks.1.hook_q_input[2],
  blocks.1.attn.hook_q[2],
  blocks.1.hook_k_input[2],
  blocks.1.attn.hook_k[2],
  blocks.1.hook_v_input[2],
  blocks.1.attn.hook_v[2],
  blocks.1.attn.hook_result[2],
  blocks.1.hook_resid_post],
 [(hook_embed, blocks.0.hook_mlp_in),
  (hook_embed, blocks.1.hook_q_input[1]),
  (hook_embed, blocks.1.hook_k_input[1]),
  (hook_embed, blocks.1.hook_v_input[1]),
  (hook_embed, blocks.1.hook_q_input[2]),
  (hook_embed, blocks.1.hook_k_input[2]),
  (hook_embed, blocks.1.hook_v_input[2]),
  (hook_embed, blocks.1.hook_resid_post),
  (hook_pos_embed, blocks.0.hook_mlp_in),
  (hook_pos_embed, blocks.1.hook_q_input[1]),
  (hook_pos_embed, blocks.1.hook_k_input[1]),
  (hook_pos_embed, blocks.1.ho

In [18]:
from iit_utils.acdc_eval import calculate_fpr_and_tpr

acdc_circuit = build_acdc_circuit_from_list_corr(corr)
result = calculate_fpr_and_tpr(acdc_circuit, gt_circuit, full_circuit)


Summary:
 - Nodes TP rate: 0.5263157894736842
 - Nodes FP rate: 0.0
 - Edges TP rate: 0.16129032258064516
 - Edges FP rate: 0.013986013986013986


In [19]:
list(acdc_circuit.edges), list(gt_circuit.edges)

([(blocks.1.attn.hook_result[2], blocks.1.hook_resid_post),
  (blocks.1.attn.hook_result[1], blocks.1.hook_resid_post),
  (blocks.1.hook_v_input[2], blocks.1.attn.hook_v[2]),
  (blocks.1.hook_v_input[1], blocks.1.attn.hook_v[1]),
  (blocks.0.hook_mlp_out, blocks.1.hook_v_input[2]),
  (blocks.0.hook_mlp_out, blocks.1.hook_v_input[1]),
  (hook_embed, blocks.0.hook_mlp_in)],
 [(hook_embed, blocks.0.hook_mlp_in),
  (hook_embed, blocks.1.hook_q_input[1]),
  (hook_embed, blocks.1.hook_k_input[1]),
  (hook_embed, blocks.1.hook_v_input[1]),
  (hook_embed, blocks.1.hook_q_input[2]),
  (hook_embed, blocks.1.hook_k_input[2]),
  (hook_embed, blocks.1.hook_v_input[2]),
  (hook_embed, blocks.1.hook_resid_post),
  (hook_pos_embed, blocks.0.hook_mlp_in),
  (hook_pos_embed, blocks.1.hook_q_input[1]),
  (hook_pos_embed, blocks.1.hook_k_input[1]),
  (hook_pos_embed, blocks.1.hook_v_input[1]),
  (hook_pos_embed, blocks.1.hook_q_input[2]),
  (hook_pos_embed, blocks.1.hook_k_input[2]),
  (hook_pos_embed, bl